In [1]:
import json
import tqdm
from flair.data import Sentence

/home/jovyan/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
2023-10-24 16:46:35.974141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 16:46:36.197612: I tensorflow/core/util/port.cc:104] oneDNN custom operations are 

In [2]:
def read_data(input_file_path):
    data = []

    with open(input_file_path) as f:
        for line in f:
            data.append(json.loads(line))

    return data

In [170]:
def write_data(output_file_path, data):
    # Open a new JSON file for writing
    with open(output_file_path, "w") as output_file:
        for data_line in data:
            output_file.write(json.dumps(data_line) + "\n")

    print(f"Data with 'Test' property added has been saved to {output_file_path}")

In [15]:
data = read_data("./squad_data_train_pos_ner.json")

In [16]:
def find_start_position(text, pos_context, answer, context):
    sentence = Sentence(text)
    tokens = []
    spaces_no = context[:answer].count(" ")

    for word in sentence:
        tokens.append(word.text)

    start_position = 0
    token_number = 0
    match = False
    no_match = -1
    for start_position in range(len(pos_context) - len(tokens)):
        k_words_context = [list(line.values())[0][0] for line in
                      pos_context[start_position:start_position + len(tokens)]]
        next_word = [list(line.values())[0][0] for line in
                      pos_context[start_position + len(tokens):start_position + len(tokens) + 1]]
        if k_words_context == tokens and (token_number + spaces_no == answer or token_number + spaces_no + 1 == answer):
            match = True
            break

        if (token_number + spaces_no == answer):
            no_match = start_position
            break
        elif next_word == ' ' and token_number + spaces_no + 1 == answer:
            no_match = start_position
            break

        token_number += len(k_words_context[0])
    if match:
        return start_position
    return no_match

In [18]:
special_characters = ["$", ",", "''", "-LRB-", "-RRB-", ".", ":", "``"]


def get_proc_pos_between_item(pos_context, context, answers):
    positions = {}
    copy_answer = []
    i = 0
    for answer in answers["answer_start"]:
        answer_position = find_start_position(answers["text"][i], pos_context, answer, context)
        if answer_position != -1:
            positions[answer_position] = answers["answer_start"][i]
            copy_answer.append(answers["answer_start"][i])
        i += 1
    pos = ""
    offset = 0
    offsets = {}
    for index in range(len(pos_context)):
        pos_text = pos_context[index][str(index)]

        if pos_text[1] in special_characters:
            pos_text[1] = "sym"
        if index in positions.keys():
            offsets[positions[index]] = offset
        pos += f"{pos_text[0]} [{pos_text[1].lower()}] "
        offset += len(f" [{pos_text[1]}] ")

    answer_starts = []

    for index in range(len(copy_answer)):
        answer = copy_answer[index]
        spaces_no = context[:answer].count(" ") + 1
        answer += offsets[answer] - spaces_no
        if pos[answer : answer + 1] == " ":
            answer += 1
        answer_starts.append(answer)

    return pos, answer_starts


In [19]:
example_id = 65

pos, answer_starts = get_proc_pos_between_item(data[example_id]['POS_context'], data[example_id]['context'],
                          data[example_id]['answers'])
pos

"The [dt] Review [nnp] of [in] Politics [nnp] was [vbd] founded [vbn] in [in] 1939 [cd] by [in] Gurian [nnp] , [sym] modeled [vbn] after [in] German [jj] Catholic [jj] journals [nns] . [sym] It [prp] quickly [rb] emerged [vbd] as [in] part [nn] of [in] an [dt] international [jj] Catholic [jj] intellectual [jj] revival [nn] , [sym] offering [vbg] an [dt] alternative [jj] vision [nn] to [in] positivist [jj] philosophy [nn] . [sym] For [in] 44 [cd] years [nns] , [sym] the [dt] Review [nnp] was [vbd] edited [vbn] by [in] Gurian [nnp] , [sym] Matthew [nnp] Fitzsimons [nnp] , [sym] Frederick [nnp] Crosson [nnp] , [sym] and [cc] Thomas [nnp] Stritch [nnp] . [sym] Intellectual [jj] leaders [nns] included [vbd] Gurian [nnp] , [sym] Jacques [nnp] Maritain [nnp] , [sym] Frank [nnp] O'Malley [nnp] , [sym] Leo [nnp] Richard [nnp] Ward [nnp] , [sym] F. [nnp] A. [nnp] Hermens [nnp] , [sym] and [cc] John [nnp] U. [nnp] Nef [nnp] . [sym] It [prp] became [vbd] a [dt] major [jj] forum [nn] for [in] polit

In [20]:
for item in tqdm.tqdm(data):
    context = item['context']
    question = item['question']
    item['context'], item['answers']['answer_start'] = get_proc_pos_between_item(item['POS_context'], context, item['answers'])

    item.pop('POS_context', None)
    item.pop('POS_question', None)
    item.pop('NER_question', None)
    item.pop('NER_context', None)

100%|██████████| 87599/87599 [00:33<00:00, 2650.11it/s]


In [154]:
write_data("./squad_data_train_pos_ner_2.json", data)

Data with 'Test' property added has been saved to ./squad_data_train_pos_ner_2.json


In [211]:
proc_data = read_data('./squad_data_validation_pos_between.json')

In [209]:
def get_updated_answer(context, answers):
    new_answers = []
    if len(answers['answer_start']) == 0:
        return

    for index, answer in enumerate(answers['text']):
        new_answer = ''
        if len(answers['answer_start']) < index + 1:
            return new_answers
        answer_start = answers['answer_start'][index]
        if answer_start != -1:
            tokenized_context = Sentence(context[answer_start:])
            no_of_words = len(Sentence(answer))
            max_length = no_of_words * 4
            if max_length > len(tokenized_context):
                max_length = len(tokenized_context)
            for token_index in range(max_length):
                if tokenized_context[token_index].text == '[' or (len(tokenized_context) > token_index+1 and tokenized_context[token_index+1].text == ']') or token_index == no_of_words * 4 - 1:
                    new_answer += tokenized_context[token_index].text
                else:
                    new_answer += tokenized_context[token_index].text + ' '
        new_answers.append(new_answer)
    return new_answers

In [202]:
example_id = 632

In [184]:
Sentence(proc_data[example_id]['context'][1083:])

Sentence[96]: "the [dt] Patriots [nnps] , [sym] Dallas [nnp] Cowboys [nnps] , [sym] and [cc] Pittsburgh [nnp] Steelers [nnps] as [in] one [cd] of [in] four [cd] teams [nns] that [wdt] have [vbp] made [vbn] eight [cd] appearances [nns] in [in] the [dt] Super [nnp] Bowl [nnp] . [sym]"

In [203]:
proc_data[example_id]['answers']

{'text': ['pre-game and halftime coverage.',
  'halftime',
  'anchor the pre-game and halftime coverage'],
 'answer_start': [487, 443]}

In [210]:
get_updated_answer(proc_data[example_id]['context'], proc_data[example_id]['answers'])

['halftime [nn] coverage [nn] . [sym] ', 'anchor [vb]']

In [212]:
for item in tqdm.tqdm(proc_data):
    item['answers']['text'] = get_updated_answer(item['context'], item['answers'])

100%|██████████| 10570/10570 [01:48<00:00, 97.08it/s] 


In [213]:
proc_data[example_id]['answers']

{'text': ['halftime [nn] coverage [nn] . [sym] ', 'anchor [vb]'],
 'answer_start': [487, 443]}

In [214]:
write_data("./squad_data_validation_pos_ner_answer_updated.json", proc_data)

Data with 'Test' property added has been saved to ./squad_data_validation_pos_ner_answer_updated.json
